In [1]:
## Imports

import pandas as pd
import numpy as np
import os
import datetime
from datetime import timedelta
from bokeh.models import HoverTool, ColumnDataSource, BoxZoomTool
from bokeh.models.layouts import Column
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from tabulate import tabulate
import talib
from pykalman import KalmanFilter
pd.set_option("display.max_rows",1000)
#init_notebook_mode()

In [2]:
## Load data (e.g. Price series, reports, equity, etc)

# obter simbolos e filenames do json
symbol_data = {}
signals = {}
reportf = pd.read_json(os.path.abspath('reportf.json'), typ='series')
symbol_list = reportf['instruments']
#reportf['notes'] = 'OLS regression'
#reportf['volatility'] = {'EUR/AUD': 0.0821, 'EUR/NZD': 0.1021} 
#reportf['strategy'] = 'Intraday OLSHR'
#reportf['instruments'] = ['EUR/AUD', 'EUR/NZD']    
reportf = reportf.rename(columns={'consecutive_losses': 'Consecutive Losses', 'consecutive_wins ': 'Consecutive Wins '})

for file in reportf['files']:
    instrument = file['symbol']
    s = instrument.replace('/', '') + '_report'
    symbol_data[instrument] = pd.io.parsers.read_csv(file['path'][0], index_col='Datetime', parse_dates=True, names=reportf['header'])   
    
    
    signals[instrument] =  pd.io.parsers.read_csv(os.path.abspath('% s.csv' % s),
                                        parse_dates=True,
                                        names=['Buy','Buy_Y', 'Sell','Sell_Y', 'Close', 'Close_Y']
                                        )
        
    signals[instrument]['Buy'] = pd.to_datetime(signals[instrument]['Buy'])
    signals[instrument]['Sell'] = pd.to_datetime(signals[instrument]['Sell'])
    signals[instrument]['Close'] = pd.to_datetime(signals[instrument]['Close'])

    symbol_data[instrument]['SMA10'] = pd.rolling_mean(symbol_data[instrument]['Close'], 10)
    symbol_data[instrument]['SMA20'] = pd.rolling_mean(symbol_data[instrument]['Close'], 20)
    symbol_data[instrument]['SMA50'] = pd.rolling_mean(symbol_data[instrument]['Close'], 50)
    symbol_data[instrument]['SMA100'] = pd.rolling_mean(symbol_data[instrument]['Close'], 100)
    symbol_data[instrument]['SMA200'] = pd.rolling_mean(symbol_data[instrument]['Close'], 200)   
 
equity =  pd.io.parsers.read_csv(os.path.abspath('% s.csv' % 'equity'),
                                        parse_dates=True, header=0,
                                        names=['Datetime','Position', 'Cash','Comission', 'Total', 'Returns', 'Equity_Curve', 'Drawdown']
                                        ).dropna()

# print(equity.head())  
equity['Datetime'] = pd.to_datetime(equity['Datetime'])
# print(equity['Datetime'])  
# equity = equity.set_index(equity['Datetime'].values)
# print(equity.head())  



/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:29: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=10).mean()
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:30: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=20).mean()
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:31: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=50).mean()
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:32: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=100).mean()
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:33: FutureWarning: pd.rolling_mean is deprecat

In [3]:
## Show stats

print(tabulate(reportf.to_frame(), tablefmt="fancy_grid"))

╒════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════╕
│ consecutive_losses │ 0                                                                                        │
├────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────┤
│ consecutive_wins   │ 0                                                                                        │
├────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────┤
│ drawdown_duration  │ 1139.0                                                                                   │
├────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────┤
│ files              │ [{'symbol': 'BTC/USD', 'path': ['histdata/BTCUSD_2017_M12.csv']}]                        │
├────────────────────┼──────────────────────────────────────────────────────────────────

In [4]:
## Process data

'''
for instrument in symbol_list:
    df = symbol_data[instrument]
    date_index = []
    #df.head()

    for ir in df.itertuples():
        date = str(ir[1]) + ' ' + str(ir[2])
        # print date
        date = datetime.datetime.strptime(date, "%Y.%m.%d %H:%M")
        date_index.append(date)
        previousdata20 = date + timedelta(minutes=-20)
        previousdata50 = date + timedelta(minutes=-50)
        previousdata100 = date + timedelta(minutes=-100)
        previousdata200 = date + timedelta(minutes=-200)
        if date.weekday() == 6:
            if previousdata20.weekday() != 6:
                ir[4] = None
            if previousdata50.weekday() != 6:
                ir[5] = None
            if previousdata100.weekday() != 6:
                ir[6] = None
            if previousdata200.weekday() != 6:
                ir[7] = None    

    symbol_data[instrument] = df.set_index([date_index])
#     print df.head()
'''

'\nfor instrument in symbol_list:\n    df = symbol_data[instrument]\n    date_index = []\n    #df.head()\n\n    for ir in df.itertuples():\n        date = str(ir[1]) + \' \' + str(ir[2])\n        # print date\n        date = datetime.datetime.strptime(date, "%Y.%m.%d %H:%M")\n        date_index.append(date)\n        previousdata20 = date + timedelta(minutes=-20)\n        previousdata50 = date + timedelta(minutes=-50)\n        previousdata100 = date + timedelta(minutes=-100)\n        previousdata200 = date + timedelta(minutes=-200)\n        if date.weekday() == 6:\n            if previousdata20.weekday() != 6:\n                ir[4] = None\n            if previousdata50.weekday() != 6:\n                ir[5] = None\n            if previousdata100.weekday() != 6:\n                ir[6] = None\n            if previousdata200.weekday() != 6:\n                ir[7] = None    \n\n    symbol_data[instrument] = df.set_index([date_index])\n#     print df.head()\n'

In [5]:
## Plot price and slope
## todo make sure indexes have the same size so plots are linked

output_notebook()
output_file("slopes.html")

plots = []
hover={}
tools={}

for instrument in symbol_list:
    df = symbol_data[instrument] 
    series = talib.LINEARREG_SLOPE(df['Close'].values, timeperiod=14)
    price_series = df['Close']

    kf = KalmanFilter(transition_matrices=[1],
                          observation_matrices=[1],
                          initial_state_mean=0,
                          initial_state_covariance=1,
                          observation_covariance=1,
                          transition_covariance=.01)

        # Use the observed values of the price to get a rolling mean
    state_means, _ = kf.filter(series)
    state_means = pd.Series(state_means.flatten())

    price_means, _ = kf.filter(price_series.values)
    price_means = pd.Series(price_means.flatten(), index=price_series.index)

    upperband, middleband, lowerband = talib.BBANDS(series, timeperiod=20, nbdevup=0.5, nbdevdn=1, matype=0)
    std = np.std(series) / 3
    kalmanupperband = std + state_means.values
    kalmanlowerband = state_means.values - std
    
    
    p1 = figure(x_axis_type="datetime")   
    p1.grid.grid_line_alpha=0.3
    p1.xaxis.axis_label = 'Date'
    p1.yaxis.axis_label = 'Price'

    hover[instrument] = HoverTool(
            tooltips=[    
                ("", "$y{1.1111}"),            
            ]
        )
    
    ## intersection by index, fix this
   
    #pct.index = series.index.map(lambda x: datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
    #print(signals[instrument]['Buy'].values)
    #buy = series.loc(axis=0)[signals[instrument]['Buy'].values,:]
    #sell = series.loc(axis=0)[signals[instrument]['Sell'].values,:]
    #close = series.loc(axis=0)[signals[instrument]['Close'].values,:]
    
    tools[instrument] = [hover[instrument], 'reset', 'box_select', 'pan','xzoom_out', 'yzoom_out', 'xwheel_zoom', 'ywheel_zoom','xbox_zoom', 'ybox_zoom']
    #print(len(buy), len(state_means), len(pct), len(signals[instrument]['Buy'].values) )
   
    p1 = figure(output_backend="webgl", tools=tools[instrument], plot_width=900, x_axis_type="datetime", title=str(instrument) + " Slopes")
    p1.line(df.index.values, series, color='#008000', legend=str(instrument)+' pct change')
    p1.line(df.index.values, state_means.values, color='purple', legend='Kalman Filter')
    #p1.triangle(signals[instrument]['Buy'].values, buy.values, color='red', legend='OPEN-BUY orders', size=7)
    #p1.inverted_triangle(signals[instrument]['Sell'].values,sell.values, color='blue', legend='OPEN-SELL orders', size=7)
    #p1.square(signals[instrument]['Close'].values, close.values, color='black', legend='CLOSE orders', size=6)
    p1.line(df.index.values, upperband, color='orange', legend='BB')
    p1.line(df.index.values, middleband, color='red', legend='SMA20')
    p1.line(df.index.values, lowerband, color='orange', legend='BB')
    p1.line(df.index.values, kalmanupperband, color='blue', legend='KalmanB') 
    p1.line(df.index.values, kalmanlowerband, color='blue', legend='KalmanB')
    plots.append(p1)  
    
    print(len(signals[instrument]),len(price_means), len(df['Close'].values))
    
    p2 = figure(output_backend="webgl", tools=tools[instrument], x_axis_type="datetime", title=str(instrument) + " Signals", plot_width=900)
    source = ColumnDataSource(data=dict(x=df.index.values, y1=df['Close'].values))
    p2.line('x', 'y1', color='olive', legend=symbol_list[0], source=source)
    p2.line(df.index.values, price_means.values, color='purple', legend='Kalman Filter')
    p2.circle('x', 'y1', fill_color='white', size=6, legend=symbol_list[0], source=source)   
    p2.triangle(signals[instrument]['Buy'].values, signals[instrument]['Buy_Y'].values, color='red', legend='OPEN-BUY orders', size=7)
    p2.inverted_triangle(signals[instrument]['Sell'].values, signals[instrument]['Sell_Y'].values, color='blue', legend='OPEN-SELL orders', size=7)
    p2.square(signals[instrument]['Close'].values, signals[instrument]['Close_Y'].values, color='black', legend='CLOSE orders', size=7)
    plots.append(p2)
    
p = Column(children=plots)
show(p)


Loading BokehJS ...

/usr/local/lib/python3.4/dist-packages/bokeh/plotting/helpers.py:535: UserWarning: BoxZoomTool,WheelZoomTool,ZoomOutTool are being repeated
  warnings.warn("%s are being repeated" % ",".join(repeated_tools))


274 5420 5420


In [6]:
output_notebook()
output_file("returns.html")

plots = []

for instrument in symbol_list:
    
    df = symbol_data[instrument]
    df2 = signals[instrument]
    
    source = ColumnDataSource(data=dict(x=df.index.values, y1=df['Close'].values, y2=equity['Returns'].values[:-2]))
    upperband, middleband, lowerband = talib.BBANDS(df['Close'].values, timeperiod=20, nbdevup=1, nbdevdn=2, matype=0)
    ema12 = talib.EMA(df['Close'].values, timeperiod=12)
    ema26 = talib.EMA(df['Close'].values, timeperiod=26)
    ema200 = talib.EMA(df['Close'].values, timeperiod=200)
    slope = talib.LINEARREG_SLOPE(df['Close'].values, timeperiod=14)  
    slopeupperband, slopemiddleband, slopelowerband = talib.BBANDS(slope, timeperiod=20, nbdevup=1, nbdevdn=1, matype=0)

    kf = KalmanFilter(transition_matrices=[1],
                          observation_matrices=[1],
                          initial_state_mean=0,
                          initial_state_covariance=1,
                          observation_covariance=1,
                          transition_covariance=.01)

        # Use the observed values of the price to get a rolling mean

    kalman, _ = kf.filter(df['Close'].values)
    kalman = kalman.flatten()
    tools = ['reset', 'box_select', 'pan','zoom_out' ,'xwheel_zoom', 'ywheel_zoom','xbox_zoom', 'ybox_zoom']

    hover = HoverTool(
            tooltips=("$y{1.1111}"))

    # p = figure(plot_width=400, plot_height=400, tools=[hover],
    #            title="Mouse over the dots
    tools_hover = [hover, 'reset', 'box_select', 'pan','zoom_out', 'xwheel_zoom', 'ywheel_zoom','xbox_zoom', 'ybox_zoom']
    p1 = figure(output_backend="webgl", tools=tools_hover, x_axis_type="datetime", title="Signals", plot_width=900)
    p1.toolbar.active_drag = None
    # dates = df.index.apply(lambda x: dt.datetime.strptime(df.index.values,"%Y.%m.%d %H:%M"))
    p1.line('x', 'y1', color='olive', legend=symbol_list[0], source=source)
    p1.circle('x', 'y1', fill_color='white', size=6, legend=symbol_list[0], source=source)
    p1.triangle(df2['Buy'].values, df2['Buy_Y'].values, color='red', legend='OPEN-BUY orders', size=7)
    p1.inverted_triangle(df2['Sell'].values, df2['Sell_Y'].values, color='blue', legend='OPEN-SELL orders', size=7)
    p1.square(df2['Close'].values, df2['Close_Y'].values, color='black', legend='CLOSE orders', size=7)
    p1.line(df.index.values, upperband, color='orange', legend='BB')
    #p1.line(df.index.values, middleband, color='red', legend='SMA20')
    p1.line(df.index.values, lowerband, color='orange', legend='BB')
    p1.line(df.index.values, ema12, color='darkblue', legend='EMA12')
    #p1.line(df.index.values, ema200, color='blue', legend='EMA200')
    p1.line(df.index.values, ema26, color='green', legend='EMA26')
    p1.line(df.index.values, kalman, color='red', legend='Kalman Filter')
    #p1.line(df.index.values, df['SMA10'].values, color='blue', legend='SMA10')
    # p1.line(df.index.values, df['SMA20'].values, color='orange', legend='SMA20')
    #p1.line(df.index.values, df['SMA50'].values, color='purple', legend='SMA50')
    #p1.line(df.index.values, df['SMA100'].values, color='brown', legend='SMA100')
    #p1.line(df.index.values, df['SMA200'].values, color='pink', legend='SMA200')
    plots.append(p1)
    
    p2 = figure(output_backend="webgl", tools=tools, x_axis_type="datetime", title="Returns", plot_width=900)
    p2.segment('x', 0, 'x','y2', color="red", source=source, legend='Returns')
    p2.circle('x', 'y2', fill_color="red", size=6, source=source, line_color='red', line_alpha=0)
    plots.append(p2)
    

    p3 = figure(output_backend="webgl", tools=tools, x_axis_type="datetime", title="Slope", plot_width=900)
    p3.line(df.index.values, slope, color='black', legend='Slope')
    p3.circle(df.index.values, slope, color='black', legend='Slope')
    p3.line(df.index.values, slopeupperband, color='orange', legend='BB')
    p3.line(df.index.values, slopemiddleband, color='red', legend='SMA20')
    p3.line(df.index.values, slopelowerband, color='orange', legend='BB')
    # p3.line('x', 'y3', color="yellow", source=source, legend='Drawdown')
    # p3.circle('x', 'y3', fill_color="yellow", size=6, source=source, line_color='yellow', line_alpha=0)
    plots.append(p3)
    
    # p2 = figure()
    # p2.circle('x', 'y2', color="olive", alpha=0.5, source=source)

    # p = gridplot([[p1], [p2]])
    # p = vplot(p1, p2)
p = Column(children=plots)
show(p)


Loading BokehJS ...

/usr/local/lib/python3.4/dist-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 5420), ('y1', 5420), ('y2', 5419)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.4/dist-packages/bokeh/plotting/helpers.py:535: UserWarning: BoxZoomTool,WheelZoomTool are being repeated
  warnings.warn("%s are being repeated" % ",".join(repeated_tools))


In [7]:
source = ColumnDataSource(data=dict(x=equity['Datetime'].values[1:], y1=equity['Equity_Curve'].values[1:], y2=equity['Drawdown'].values[1:]))
print(equity.head())
# print(equity['Equity_Curve'].values[1:])
# print(equity['Drawdown'].values[1:])
# print equity['Datetime']
tools = ['reset', 'box_select', 'pan','xzoom_out', 'yzoom_out', 'xwheel_zoom', 'ywheel_zoom','xbox_zoom', 'ybox_zoom']

p1 = figure(output_backend="webgl", tools=tools, x_axis_type="datetime", title="Equity Curve", plot_width=900)
# dates = df.index.apply(lambda x: dt.datetime.strptime(df.index.values,"%Y.%m.%d %H:%M"))
p1.line('x', 'y1', color='blue', legend='Equity Curve', source=source)
p1.circle('x', 'y1', fill_color='blue', size=6, legend='Equity Curve', source=source)


p2 = figure(output_backend="webgl", tools=tools, x_axis_type="datetime", title="Drawdown", plot_width=900)
p2.segment('x', 0, 'x','y2', color="red", source=source, legend='Drawdown')
p2.circle('x', 'y2', fill_color="red", size=6, source=source, line_color='red', line_alpha=0)

# p2 = figure(webgl=True, tools=tools, x_axis_type="datetime")
# p2.line('x', 'y2', color="red", source=source, legend='Drawdown')
# p2.circle('x', 'y2', fill_color="red", size=6, source=source, line_color='red', line_alpha=0)
output_file("equity.html")

p = Column(children=[p1,p2])
show(p)
# factors = ["a", "b", "c", "d", "e", "f", "g", "h"]
# x =  [50, 40, 65, 10, 25, 37, 80, 60]

# dot = figure(title="Categorical Dot Plot", tools="", toolbar_location=None,
#             y_range=factors, x_range=[0,100])

# dot.segment(0, factors, x, factors, line_width=2, line_color="green", )
# dot.circle(x, factors, size=15, fill_color="orange", line_color="green", line_width=3, )

# 

             Datetime  Position   Cash  Comission  Total  Returns  \
1 2017-12-16 00:00:00       0.0  600.0        0.0  600.0      0.0   
2 2017-12-16 00:05:00       0.0  600.0        0.0  600.0      0.0   
3 2017-12-16 00:10:00       0.0  600.0        0.0  600.0      0.0   
4 2017-12-16 00:15:00       0.0  600.0        0.0  600.0      0.0   
5 2017-12-16 00:20:00       0.0  600.0        0.0  600.0      0.0   

   Equity_Curve  Drawdown  
1           1.0       0.0  
2           1.0       0.0  
3           1.0       0.0  
4           1.0       0.0  
5           1.0       0.0  


/usr/local/lib/python3.4/dist-packages/bokeh/plotting/helpers.py:535: UserWarning: BoxZoomTool,WheelZoomTool,ZoomOutTool are being repeated
  warnings.warn("%s are being repeated" % ",".join(repeated_tools))
